# Notebook Title

## Setup Python and R environment
you can ignore this section

In [2]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [3]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## 👉 download your data

You can write code here to download your dataset. Or if you already have it, just leave the URL in the comments and just load it into a pandas or R (or both) dataframe.

In [9]:
df = pd.read_csv('raw_data/2023_raw_open.csv')
pd.set_option('display.max_colwidth', None)
df.sample(10)

,Month,Borough,Equipment Type,Equipment Code,Total Outages,Scheduled Outages,Unscheduled Outages,Entrapments,Time Since Major Improvement,AM Peak Availability,...,PM Peak Availability,PM Peak Hours Available,PM Peak Total Hours,24-Hour Availability,24-Hour Hours Available,24-Hour Total Hours,Station Name,Station MRN,Station Complex Name,Station Complex MRN
2944,09/01/2023,Manhattan,Elevator,EL443,4,2,2,0,81.0,0.991667,...,1.000000,120.000000,120.0,0.983866,708.383333,720.0,LEXINGTONAV/63ST-63S-F/Q,223,Lexington Av/63 St - Station,223
466,02/01/2023,Queens,Elevator,EL412,2,1,1,0,14.0,1.000000,...,1.000000,112.000000,112.0,0.991220,666.100000,672.0,JAMAICACENTER-PARSONS/ARCHER-QBL-E/J/Z,278,Jamaica Center-Parsons/Archer - Station,278
1456,05/01/2023,Brooklyn,Elevator,EL307,1,1,0,0,231.0,1.000000,...,1.000000,124.000000,124.0,0.993728,739.333333,744.0,ATLANTICAV-BARCLAYSCTR-BWY-B/Q,40,"Atlantic Av (B,Q,2,3,4,5)/Pacific St (D,N,R)",617
1754,06/01/2023,Manhattan,Elevator,EL295X,0,0,0,0,NaN,1.000000,...,1.000000,120.000000,120.0,1.000000,720.000000,720.0,42ST-BRYANTPK-6AV-B/D/F/M,226,"42 St-Bryant Pk (B,D,F,M)/5 Av (7)",609
3036,09/01/2023,Manhattan,Elevator,EL13X,0,0,0,0,NaN,1.000000,...,0.982500,117.900000,120.0,0.987130,710.733333,720.0,CORTLANDTST-7AV-1,328,Cortlandt St - Station,328
3335,10/01/2023,Manhattan,Elevator,EL245,5,3,2,0,223.0,0.967742,...,1.000000,124.000000,124.0,0.968817,720.800000,744.0,LEXINGTONAV/53ST-QBL-E/M,275,"Lexington Av-53 St (E,M)/51 St (6)",612
241,01/01/2023,Queens,Elevator,EL428,5,1,4,1,203.0,0.947043,...,0.936156,116.083333,124.0,0.958423,713.066667,744.0,QUEENSPLAZA-QBL-E/M/R,273,Queens Plaza - Station,273
3810,11/01/2023,Manhattan,Elevator,EL16X,0,0,0,0,NaN,1.000000,...,1.000000,120.000000,120.0,1.000000,720.000000,720.0,CORTLANDTST-BWY-R,21,"Chambers St (A,C)/WTC (E)/Park Place (2,3)/Cortlandt St (R, W)",624
4016,12/01/2023,Bronx,Elevator,EL128,5,2,3,0,89.0,0.951613,...,0.985887,122.250000,124.0,0.968974,720.916667,744.0,SIMPSONST-LWP-2/5,430,Simpson St - Station,430
1076,04/01/2023,Manhattan,Elevator,EL230,4,2,2,2,239.0,0.947361,...,1.000000,120.000000,120.0,0.983657,708.233333,720.0,TIMESSQ-42ST-BWY-N/R/Q,11,"Times Sq-42 St (N,Q,R,S,1,2,3,7)/42 St (A,C,E)",611


In [10]:
%%R
df <- read_csv('raw_data/2023_raw_open.csv')

Rows: 4200 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (7): Month, Borough, Equipment Type, Equipment Code, Time Since Major I...
dbl (15): Total Outages, Scheduled Outages, Unscheduled Outages, Entrapments...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [12]:
df.shape

(4200, 22)

## 👉 convert addresses --> lat/long 

See the [census-examples](https://github.com/data4news/census-examples) repository for examples. If you need help, try asking in the class slack channel. Chances are someone in the class is struggling with the same problem as you are so we might as well all learn together in the same slack channel! 

In [11]:
# load the geostation file
df_geostation = pd.read_csv('raw_data/subway_stations.csv')

# merge the geostation file with the df_ele
df_merged = df.merge(df_geostation, left_on='Station Complex MRN', right_on='Complex ID')

pd.set_option('display.max_columns', 500)

# rename the columns
df_merged = df_merged.rename(columns={'GTFS Latitude': 'lat',  'GTFS Longitude': 'long'})


In [14]:
# check the merged file 

# save the merged file
df_merged.to_csv('2023_ele_latlong.csv', index=False)
print(df_merged.shape)
df_merged.sample(5)


# WHY IS THERE MORE ROWS THAN THE ORIGINAL FILE??

(6936, 38)


,Month,Borough_x,Equipment Type,Equipment Code,Total Outages,Scheduled Outages,Unscheduled Outages,Entrapments,Time Since Major Improvement,AM Peak Availability,AM Peak Hours Available,AM Peak Total Hours,PM Peak Availability,PM Peak Hours Available,PM Peak Total Hours,24-Hour Availability,24-Hour Hours Available,24-Hour Total Hours,Station Name,Station MRN,Station Complex Name,Station Complex MRN,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough_y,Daytime Routes,Structure,lat,long,North Direction Label,South Direction Label,ADA,ADA Notes,Georeference
3312,06/01/2023,Manhattan,Elevator,EL716,7,3,4,0,103.0,0.922222,110.666667,120.0,0.925278,111.033333,120.0,0.902083,649.500000,720.0,FULTONST-LEX-4/5,412,"Fulton St (A,C,J,Z,2,3,4,5)",628,106,628,M22,BMT,Jamaica,Fulton St,M,J Z,Subway,40.710374,-74.007582,Brooklyn,Broad St,1,NaN,POINT (-74.007582 40.710374)
5967,11/01/2023,Brooklyn,Elevator,EL708,1,1,0,0,162.0,1.000000,120.000000,120.0,1.000000,120.000000,120.0,0.992824,714.833333,720.0,JAYST-METROTECH-8AV-A/C/F,174,"Jay St-MetroTech (A,C,F,R)",636,174,636,A41,IND,8th Av - Fulton St,Jay St-MetroTech,Bk,A C F,Subway,40.692338,-73.987342,Manhattan,Euclid - Lefferts - Rockaways - Coney Island,1,NaN,POINT (-73.987342 40.692338)
4640,09/01/2023,Manhattan,Elevator,EL335,1,1,0,0,221.0,1.000000,120.000000,120.0,1.000000,120.000000,120.0,0.993356,715.216667,720.0,W4ST-WASHSQ-8AV-A/B/C/D/E/F/M,167,W 4 St-Wash Sq - Station,167,167,167,D20,IND,6th Av - Culver,W 4 St-Wash Sq,M,B D F M,Subway,40.732338,-74.000495,Uptown - Queens,Downtown & Brooklyn,1,NaN,POINT (-74.000495 40.732338)
1716,03/01/2023,Brooklyn,Elevator,EL378,1,1,0,0,128.0,1.000000,124.000000,124.0,0.987903,122.500000,124.0,0.996640,741.500000,744.0,BAYPKWY-WST-D,68,Bay Pkwy - Station,68,68,68,B21,BMT,West End,Bay Pkwy,Bk,D,Elevated,40.601875,-73.993728,Manhattan,Coney Island,1,NaN,POINT (-73.993728 40.601875)
5640,10/01/2023,Brooklyn,Elevator,EL304,3,2,1,1,234.0,1.000000,124.000000,124.0,0.975806,121.000000,124.0,0.986358,733.850000,744.0,ATLANTICAV-BARCLAYSCTR-EPK-2/3/4/5,338,"Atlantic Av (B,Q,2,3,4,5)/Pacific St (D,N,R)",617,27,617,R31,BMT,4th Av,Atlantic Av-Barclays Ctr,Bk,D N R,Subway,40.683666,-73.978810,Manhattan,Coney Island - Bay Ridge,1,NaN,POINT (-73.97881 40.683666)


## 👉 convert lat/long to census geography codes 

(like 'GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK', etc...)

Same note as above, see [census-examples](https://github.com/data4news/census-examples) repository for examples or ask in the class slack channel if stuck.

In [15]:
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

import requests_cache
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng):
    try:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        params = {
            "x": lng,
            "y": lat,
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        response = cache.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        census = data['result']['geographies']['Census Blocks'][0]
        return census
    except Exception as e:
        print(f"Error geocoding ({lat}, {lng}): {e}")
        return None

def bulk_geocode(latitudes, longitudes):
    """
    Geocode a list of latitudes and longitudes in parallel (for speed).
    """

    with ThreadPoolExecutor() as tpe:
        latitudes = df_merged['lat']
        longitudes = df_merged['long']
        mapped_results = tpe.map(geocode, latitudes, longitudes)
        data = list(tqdm(mapped_results, total=len(df_merged)))

    return pd.DataFrame(data)

census_geos_df = bulk_geocode(df_merged['lat'], df_merged['long']) 
census_geos_df.head()

  0%|          | 0/6936 [00:00<?, ?it/s]

,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,INTPTLAT,FUNCSTAT,NAME,OBJECTID,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,170,360470039003001,+40.6839070,3001,0,36,3001,210701004656723,BK,+40.6839070,S,Block 3001,6255754,003900,-073.9801517,3,17988,81,-073.9801517,G5040,L,U,047
1,,0,360470035001003,+40.6845105,1003,0,36,1003,210701004649659,BK,+40.6845105,S,Block 1003,4708593,003500,-073.9768500,1,21849,0,-073.9768500,G5040,L,U,047
2,,0,360470035001003,+40.6845105,1003,0,36,1003,210701004649659,BK,+40.6845105,S,Block 1003,4708593,003500,-073.9768500,1,21849,0,-073.9768500,G5040,L,U,047
3,,757,360470011001011,+40.6915136,1011,0,36,1011,210701004654809,BK,+40.6915136,S,Block 1011,5851993,001100,-073.9857897,1,12271,406,-073.9857897,G5040,L,U,047
4,,0,360470011001001,+40.6936303,1001,0,36,1001,210701004648557,BK,+40.6936303,S,Block 1001,5848934,001100,-073.9880227,1,40946,0,-073.9880227,G5040,L,U,047


In [16]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK']
census_geos_df = census_geos_df[to_keep]
census_geos_df

,GEOID,STATE,COUNTY,TRACT,BLOCK
0,360470039003001,36,047,003900,3001
1,360470035001003,36,047,003500,1003
2,360470035001003,36,047,003500,1003
3,360470011001011,36,047,001100,1011
4,360470011001001,36,047,001100,1001
...,...,...,...,...,...
6931,360610094002007,36,061,009400,2007
6932,360610269004001,36,061,026900,4001
6933,360610092002008,36,061,009200,2008
6934,360610080001000,36,061,008000,1000


## 👉 Output Data

Output your dataframe containing your data and the Census connector codes (like tract, block, etc...).

In [17]:
df_with_geos = pd.concat(
    [ 
        df_merged.reset_index(drop=True),
        census_geos_df.reset_index(drop=True)
    ], 
    axis=1)

df_with_geos.head()

,Month,Borough_x,Equipment Type,Equipment Code,Total Outages,Scheduled Outages,Unscheduled Outages,Entrapments,Time Since Major Improvement,AM Peak Availability,AM Peak Hours Available,AM Peak Total Hours,PM Peak Availability,PM Peak Hours Available,PM Peak Total Hours,24-Hour Availability,24-Hour Hours Available,24-Hour Total Hours,Station Name,Station MRN,Station Complex Name,Station Complex MRN,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough_y,Daytime Routes,Structure,lat,long,North Direction Label,South Direction Label,ADA,ADA Notes,Georeference,GEOID,STATE,COUNTY,TRACT,BLOCK
0,01/01/2023,Brooklyn,Elevator,EL301,4,2,2,2,225.0,0.978629,121.35,124.0,0.944624,117.133333,124.0,0.977957,727.600000,744.0,ATLANTICAV-BARCLAYSCTR-4AV-D/N/R,27,"Atlantic Av (B,Q,2,3,4,5)/Pacific St (D,N,R)",617,27,617,R31,BMT,4th Av,Atlantic Av-Barclays Ctr,Bk,D N R,Subway,40.683666,-73.978810,Manhattan,Coney Island - Bay Ridge,1,NaN,POINT (-73.97881 40.683666),360470039003001,36,047,003900,3001
1,01/01/2023,Brooklyn,Elevator,EL301,4,2,2,2,225.0,0.978629,121.35,124.0,0.944624,117.133333,124.0,0.977957,727.600000,744.0,ATLANTICAV-BARCLAYSCTR-4AV-D/N/R,27,"Atlantic Av (B,Q,2,3,4,5)/Pacific St (D,N,R)",617,40,617,D24,BMT,Broadway - Brighton,Atlantic Av-Barclays Ctr,Bk,B Q,Subway,40.684460,-73.976890,Manhattan,Brighton Beach & Coney Island,1,NaN,POINT (-73.97689 40.68446),360470035001003,36,047,003500,1003
2,01/01/2023,Brooklyn,Elevator,EL301,4,2,2,2,225.0,0.978629,121.35,124.0,0.944624,117.133333,124.0,0.977957,727.600000,744.0,ATLANTICAV-BARCLAYSCTR-4AV-D/N/R,27,"Atlantic Av (B,Q,2,3,4,5)/Pacific St (D,N,R)",617,338,617,235,IRT,Eastern Pky,Atlantic Av-Barclays Ctr,Bk,2 3 4 5,Subway,40.684359,-73.977666,Manhattan,Flatbush - New Lots,1,NaN,POINT (-73.977666 40.684359),360470035001003,36,047,003500,1003
3,01/01/2023,Brooklyn,Elevator,EL709,6,2,4,0,145.0,0.985081,122.15,124.0,0.974866,120.883333,124.0,0.971214,722.583333,744.0,JAYST-METROTECH-BWY-R,25,"Jay St-MetroTech (A,C,F,R)",636,25,636,R29,BMT,Broadway,Jay St-MetroTech,Bk,R,Subway,40.692180,-73.985942,Manhattan,Bay Ridge - 95 St,1,NaN,POINT (-73.985942 40.69218),360470011001011,36,047,001100,1011
4,01/01/2023,Brooklyn,Elevator,EL709,6,2,4,0,145.0,0.985081,122.15,124.0,0.974866,120.883333,124.0,0.971214,722.583333,744.0,JAYST-METROTECH-BWY-R,25,"Jay St-MetroTech (A,C,F,R)",636,174,636,A41,IND,8th Av - Fulton St,Jay St-MetroTech,Bk,A C F,Subway,40.692338,-73.987342,Manhattan,Euclid - Lefferts - Rockaways - Coney Island,1,NaN,POINT (-73.987342 40.692338),360470011001001,36,047,001100,1001


In [18]:
df_with_geos.to_csv('2023_subway_censusgeo.csv', index=False)